# Data Wrangling

## Imports

In [48]:
import pandas as pd

from pathlib import Path
from sklearn.impute import SimpleImputer, KNNImputer
from zipfile import ZipFile
from joblib import dump

## ZIP Import

In [49]:
source_df = pd.read_csv(ZipFile('data/original/immoscout_v2.zip').open("immo_data_202208_v2.csv"), sep=',', index_col = 0, low_memory=False)
clean_df = source_df.copy()

### Delete Unnamed Columns

In [50]:
clean_df = clean_df.loc[:, ~clean_df.columns.str.contains('^Unnamed')]
clean_df.head()

,Municipality,Living space,Plot area,Floor space,Availability,location,description,detailed_description,url,table,...,features,description_detailed,Floor space:,Number of floors:,Volume:,plz,Number of toilets:,Gross yield:,Minimum floor space:,space_cleaned
0,Biberstein,100 m²,NaN,NaN,On request,"5023 Biberstein, AG","3.5 rooms, 100 m²«Luxuriöse Attika-Wohnung mit...",DescriptionLuxuriöse Attika-Wohnung direkt an ...,https://www.immoscout24.ch//en/d/penthouse-buy...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5023.0,NaN,NaN,NaN,NaN
1,Biberstein,156 m²,222 m²,242 m²,On request,"Buhldenstrasse 8d5023 Biberstein, AG","4.5 rooms, 156 m²«Stilvolle Liegenschaft - ruh...",DescriptionStilvolle Liegenschaft an ruhiger L...,https://www.immoscout24.ch//en/d/terrace-house...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5023.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,"5022 Rombach, AG","2.5 rooms, 93 m²«Moderne, lichtdurchflutete At...","detail_responsive#description_title2,5 Zimmerw...",https://www.immoscout24.ch//en/d/penthouse-buy...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5022.0,NaN,NaN,NaN,NaN
3,Biberstein,154 m²,370 m²,257 m²,On request,"Buhaldenstrasse 8A5023 Biberstein, AG","4.5 rooms, 154 m²«AgentSelly - Luxuriöses Eckh...",DescriptionDieses äusserst grosszügige Minergi...,https://www.immoscout24.ch//en/d/detached-hous...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5023.0,NaN,NaN,NaN,NaN
4,Küttigen,142 m²,NaN,NaN,On request,"5022 Rombach, AG","4.5 rooms, 142 m²«MIT GARTENSITZPLATZ UND VIEL...",DescriptionAus ehemals zwei Wohnungen wurde ei...,https://www.immoscout24.ch//en/d/flat-buy-romb...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5022.0,NaN,NaN,NaN,NaN


### There are Column which we can merge

In [51]:
clean_df['Municipality']    = clean_df['Municipality'].fillna(clean_df['detail_responsive#municipality'])
clean_df['Municipality']    = clean_df['Municipality'].fillna(clean_df['Gemeinde'])
clean_df['Municipality']    = clean_df['Municipality'].fillna(clean_df['Commune'])
clean_df['Municipality']    = clean_df['Municipality'].fillna(clean_df['Comune'])
clean_df['Municipality']    = clean_df['Municipality'].fillna(clean_df['Municipality_merged'])

clean_df['Living space']    = clean_df['Living space'].fillna(clean_df['detail_responsive#surface_living'])
clean_df['Living space']    = clean_df['Living space'].fillna(clean_df['Wohnfläche'])
clean_df['Living space']    = clean_df['Living space'].fillna(clean_df['Surface habitable'])
clean_df['Living space']    = clean_df['Living space'].fillna(clean_df['Superficie abitabile'])
clean_df['Living space']    = clean_df['Living space'].fillna(clean_df['Living_space_merged'])
clean_df['Living space']    = clean_df['Living space'].fillna(clean_df['Living_area_unified'])
clean_df['Living space']    = clean_df['Living space'].fillna(clean_df['Space extracted'])
clean_df['Living space']    = clean_df['Living space'].fillna(clean_df['Surface living:'])

clean_df['Plot area']       = clean_df['Plot area'].fillna(clean_df['detail_responsive#surface_property'])
clean_df['Plot area']       = clean_df['Plot area'].fillna(clean_df['Grundstücksfläche'])
clean_df['Plot area']       = clean_df['Plot area'].fillna(clean_df['Surface du terrain'])
clean_df['Plot area']       = clean_df['Plot area'].fillna(clean_df['Superficie del terreno'])
clean_df['Plot area']       = clean_df['Plot area'].fillna(clean_df['Plot_area_merged'])
clean_df['Plot area']       = clean_df['Plot area'].fillna(clean_df['Plot_area_unified'])
clean_df['Plot area']       = clean_df['Plot area'].fillna(clean_df['Land area:'])

clean_df['Floor space']     = clean_df['Floor space'].fillna(clean_df['detail_responsive#surface_usable'])
clean_df['Floor space']     = clean_df['Floor space'].fillna(clean_df['Nutzfläche'])
clean_df['Floor space']     = clean_df['Floor space'].fillna(clean_df['Surface utile'])
clean_df['Floor space']     = clean_df['Floor space'].fillna(clean_df['Superficie utile'])
clean_df['Floor space']     = clean_df['Floor space'].fillna(clean_df['Floor_space_merged'])
clean_df['Floor space']     = clean_df['Floor space'].fillna(clean_df['Floor space:'])

clean_df['Floor']           = clean_df['Floor'].fillna(clean_df['detail_responsive#floor'])
clean_df['Floor']           = clean_df['Floor'].fillna(clean_df['Stockwerk'])
clean_df['Floor']           = clean_df['Floor'].fillna(clean_df['Étage'])
clean_df['Floor']           = clean_df['Floor'].fillna(clean_df['Piano'])
clean_df['Floor']           = clean_df['Floor'].fillna(clean_df['Floor_merged'])

clean_df['Availability']  = clean_df['Availability'].fillna(clean_df['detail_responsive#available_from'])
clean_df['Availability']  = clean_df['Availability'].fillna(clean_df['Verfügbarkeit'])
clean_df['Availability']  = clean_df['Availability'].fillna(clean_df['Disponibilité'])
clean_df['Availability']  = clean_df['Availability'].fillna(clean_df['Disponibilità'])
clean_df['Availability']  = clean_df['Availability'].fillna(clean_df['Availability_merged'])

clean_df = clean_df.drop(['detail_responsive#municipality', 'Gemeinde', 'Commune', 'Comune', 'Municipality_merged',
                            'detail_responsive#surface_living', 'Wohnfläche', 'Surface habitable', 'Superficie abitabile', 'Living_space_merged', 'Living_area_unified', 'Space extracted', 'Surface living:',
                            'detail_responsive#surface_property', 'Grundstücksfläche', 'Surface du terrain', 'Superficie del terreno', 'Plot_area_merged', 'Plot_area_unified', 'Land area:',
                            'detail_responsive#surface_usable', 'Nutzfläche', 'Surface utile', 'Superficie utile', 'Floor_space_merged', 'Floor space:',
                            'detail_responsive#floor', 'Stockwerk', 'Étage', 'Piano', 'Floor_merged',
                            'detail_responsive#available_from', 'Verfügbarkeit', 'Disponibilité', 'Disponibilità', 'Availability_merged'], axis=1)

clean_df.head()

,Municipality,Living space,Plot area,Floor space,Availability,location,description,detailed_description,url,table,...,Year built:,features,description_detailed,Number of floors:,Volume:,plz,Number of toilets:,Gross yield:,Minimum floor space:,space_cleaned
0,Biberstein,100 m²,NaN,NaN,On request,"5023 Biberstein, AG","3.5 rooms, 100 m²«Luxuriöse Attika-Wohnung mit...",DescriptionLuxuriöse Attika-Wohnung direkt an ...,https://www.immoscout24.ch//en/d/penthouse-buy...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5023.0,NaN,NaN,NaN,NaN
1,Biberstein,156 m²,222 m²,242 m²,On request,"Buhldenstrasse 8d5023 Biberstein, AG","4.5 rooms, 156 m²«Stilvolle Liegenschaft - ruh...",DescriptionStilvolle Liegenschaft an ruhiger L...,https://www.immoscout24.ch//en/d/terrace-house...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5023.0,NaN,NaN,NaN,NaN
2,Küttigen,93 m²,NaN,NaN,Immediately,"5022 Rombach, AG","2.5 rooms, 93 m²«Moderne, lichtdurchflutete At...","detail_responsive#description_title2,5 Zimmerw...",https://www.immoscout24.ch//en/d/penthouse-buy...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5022.0,NaN,NaN,NaN,NaN
3,Biberstein,154 m²,370 m²,257 m²,On request,"Buhaldenstrasse 8A5023 Biberstein, AG","4.5 rooms, 154 m²«AgentSelly - Luxuriöses Eckh...",DescriptionDieses äusserst grosszügige Minergi...,https://www.immoscout24.ch//en/d/detached-hous...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5023.0,NaN,NaN,NaN,NaN
4,Küttigen,142 m²,NaN,NaN,On request,"5022 Rombach, AG","4.5 rooms, 142 m²«MIT GARTENSITZPLATZ UND VIEL...",DescriptionAus ehemals zwei Wohnungen wurde ei...,https://www.immoscout24.ch//en/d/flat-buy-romb...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5022.0,NaN,NaN,NaN,NaN


### Take Informations from Column "details", and put the Informations into "rooms" and "Living space" if they are nan or 0

In [52]:
clean_df['rooms_from_details'] = (clean_df['details'].str.extract(r'(\d+) rooms')).astype(float)
clean_df['space_from_details'] = (clean_df['details'].str.extract(r'(\d+) m²')).astype(float)

clean_df['rooms'] = clean_df['rooms'].mask(clean_df['rooms'] == 0.0, clean_df['rooms_from_details'])
clean_df['rooms'] = clean_df['rooms'].fillna(clean_df['rooms_from_details'])

clean_df['Living space'] = clean_df['Living space'].fillna(clean_df['space_from_details'])

clean_df = clean_df.drop(['details', 'rooms_from_details', 'space_from_details'], axis=1)

clean_df.head()

,Municipality,Living space,Plot area,Floor space,Availability,location,description,detailed_description,url,table,...,Year built:,features,description_detailed,Number of floors:,Volume:,plz,Number of toilets:,Gross yield:,Minimum floor space:,space_cleaned
0,Biberstein,100 m²,NaN,NaN,On request,"5023 Biberstein, AG","3.5 rooms, 100 m²«Luxuriöse Attika-Wohnung mit...",DescriptionLuxuriöse Attika-Wohnung direkt an ...,https://www.immoscout24.ch//en/d/penthouse-buy...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5023.0,NaN,NaN,NaN,NaN
1,Biberstein,156 m²,222 m²,242 m²,On request,"Buhldenstrasse 8d5023 Biberstein, AG","4.5 rooms, 156 m²«Stilvolle Liegenschaft - ruh...",DescriptionStilvolle Liegenschaft an ruhiger L...,https://www.immoscout24.ch//en/d/terrace-house...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5023.0,NaN,NaN,NaN,NaN
2,Küttigen,93 m²,NaN,NaN,Immediately,"5022 Rombach, AG","2.5 rooms, 93 m²«Moderne, lichtdurchflutete At...","detail_responsive#description_title2,5 Zimmerw...",https://www.immoscout24.ch//en/d/penthouse-buy...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5022.0,NaN,NaN,NaN,NaN
3,Biberstein,154 m²,370 m²,257 m²,On request,"Buhaldenstrasse 8A5023 Biberstein, AG","4.5 rooms, 154 m²«AgentSelly - Luxuriöses Eckh...",DescriptionDieses äusserst grosszügige Minergi...,https://www.immoscout24.ch//en/d/detached-hous...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5023.0,NaN,NaN,NaN,NaN
4,Küttigen,142 m²,NaN,NaN,On request,"5022 Rombach, AG","4.5 rooms, 142 m²«MIT GARTENSITZPLATZ UND VIEL...",DescriptionAus ehemals zwei Wohnungen wurde ei...,https://www.immoscout24.ch//en/d/flat-buy-romb...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5022.0,NaN,NaN,NaN,NaN


### Remove m² from Columns

In [53]:
clean_df['Living space'] = clean_df['Living space'].astype(str)
clean_df['Living space'] = clean_df['Living space'].str.replace('m²', '')
clean_df['Living space'] = clean_df['Living space'].str.replace('m2', '')
clean_df['Living space'] = clean_df['Living space'].astype(float)

clean_df['Plot area'] = clean_df['Plot area'].astype(str)
clean_df['Plot area'] = clean_df['Plot area'].str.replace('m²', '')
clean_df['Plot area'] = clean_df['Plot area'].str.replace(',', '')
clean_df['Plot area'] = clean_df['Plot area'].astype(float)

clean_df['Floor space'] = clean_df['Floor space'].astype(str)
clean_df['Floor space'] = clean_df['Floor space'].str.replace('m²', '')
clean_df['Floor space'] = clean_df['Floor space'].str.replace('m2', '')
clean_df['Floor space'] = clean_df['Floor space'].astype(float)

clean_df['Floor'] = clean_df['Floor'].astype(str)
clean_df['Floor'] = clean_df['Floor'].str.replace('Ground floor', '0')
clean_df['Floor'] = clean_df['Floor'].str.replace('GF', '0')
clean_df['Floor'] = clean_df['Floor'].str.replace('. floor', '', regex=False)
clean_df['Floor'] = clean_df['Floor'].str.replace(r'(\d+)\. Basement', lambda x: str(int(x.group(1)) * -1), regex=True)
clean_df['Floor'] = clean_df['Floor'].astype(float)

clean_df['rooms'] = clean_df['rooms'].astype(str)
clean_df['rooms'] = clean_df['rooms'].str.replace('rm', '')
clean_df['rooms'] = clean_df['rooms'].astype(float)

clean_df.head()

,Municipality,Living space,Plot area,Floor space,Availability,location,description,detailed_description,url,table,...,Year built:,features,description_detailed,Number of floors:,Volume:,plz,Number of toilets:,Gross yield:,Minimum floor space:,space_cleaned
0,Biberstein,100.0,NaN,NaN,On request,"5023 Biberstein, AG","3.5 rooms, 100 m²«Luxuriöse Attika-Wohnung mit...",DescriptionLuxuriöse Attika-Wohnung direkt an ...,https://www.immoscout24.ch//en/d/penthouse-buy...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5023.0,NaN,NaN,NaN,NaN
1,Biberstein,156.0,222.0,242.0,On request,"Buhldenstrasse 8d5023 Biberstein, AG","4.5 rooms, 156 m²«Stilvolle Liegenschaft - ruh...",DescriptionStilvolle Liegenschaft an ruhiger L...,https://www.immoscout24.ch//en/d/terrace-house...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5023.0,NaN,NaN,NaN,NaN
2,Küttigen,93.0,NaN,NaN,Immediately,"5022 Rombach, AG","2.5 rooms, 93 m²«Moderne, lichtdurchflutete At...","detail_responsive#description_title2,5 Zimmerw...",https://www.immoscout24.ch//en/d/penthouse-buy...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5022.0,NaN,NaN,NaN,NaN
3,Biberstein,154.0,370.0,257.0,On request,"Buhaldenstrasse 8A5023 Biberstein, AG","4.5 rooms, 154 m²«AgentSelly - Luxuriöses Eckh...",DescriptionDieses äusserst grosszügige Minergi...,https://www.immoscout24.ch//en/d/detached-hous...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5023.0,NaN,NaN,NaN,NaN
4,Küttigen,142.0,NaN,NaN,On request,"5022 Rombach, AG","4.5 rooms, 142 m²«MIT GARTENSITZPLATZ UND VIEL...",DescriptionAus ehemals zwei Wohnungen wurde ei...,https://www.immoscout24.ch//en/d/flat-buy-romb...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5022.0,NaN,NaN,NaN,NaN


### How many NAN's has each column

In [54]:
(clean_df.isnull().sum() / len(clean_df)) * 100

Municipality             1.023086
Living space             2.668920
Plot area               63.480272
Floor space             74.609670
Availability            40.594280
                          ...    
plz                      0.075619
Number of toilets:      99.675281
Gross yield:            99.937725
Minimum floor space:    99.991104
space_cleaned           59.405720
Length: 95, dtype: float64

### Remove redundant columns

In [55]:
columns = ['Municipality',#unbrauchbar wegen long und lat
            'Availability',#zu wenig Daten
            'location',#long und lat
            'location_parsed',#long und lat
            'description',#unbrauchbar         
            'detailed_description',#unbrauchbar
            'url',#unbrauchbar
            'table',#unbrauchbar
            'Gross return',#zu wenig Daten
            'title',#unbrauchbar
            'address',#unbrauchbar wegen long und lat
            'price',#gibt price_cleaned
            'link',#unbrauchbar
            'details_structured',#unbrauchbar
            'lat',#gibt latitute
            'lon',#gibt longitude
            'index',#unbrauchbar
            'Locality',#long und lat
            'plz_parsed',#gibt ZIP
            'type',#gibt type_unified
            'Floor_unified',#gibt Floor
            'provider',#unbrauchbar
            'space',#gibt Living space
            'price_s',#gibt price_cleaned
            'address_s',#unbrauchbar wegen long und lat
            'No. of rooms:',#gibt schon rooms
            'Number of apartments:',#unbrauchbar
            'Room height:',#zu wenig Daten
            'Last refurbishment:',#zu wenig Daten
            'Year built:',#zu wenig Daten
            'features',#unbrauchbar
            'description_detailed',#unbrauchbar
            'Number of floors:',#zu wenig Daten
            'Volume:',#unbrauchbar
            'plz',#gibt ZIP
            'Number of toilets:',#zu wenig Daten
            'Gross yield:',#zu wenig Daten
            'Minimum floor space:',#zu wenig Daten
            'space_cleaned',#gibt Living space
            'gde_politics_bdp',#zu wenig Daten
            'gde_politics_evp',#zu wenig Daten
            'gde_politics_glp',#zu wenig Daten
            'gde_politics_pda',#zu wenig Daten
            'gde_politics_rights'#zu wenig Daten
]

clean_df = clean_df.drop(columns, axis=1)
clean_df.head()

,Living space,Plot area,Floor space,Floor,ForestDensityL,ForestDensityM,ForestDensityS,Latitude,Longitude,NoisePollutionRailwayL,...,gde_private_apartments,gde_social_help_quota,gde_tax,gde_workers_sector1,gde_workers_sector2,gde_workers_sector3,gde_workers_total,price_cleaned,rooms,type_unified
0,100.0,NaN,NaN,4.0,0.511176,0.286451,0.090908,47.415927,8.085840,0.0,...,686.0,2.234259,5.89,14.0,9.0,308.0,331.0,1150000.0,5.0,penthouse
1,156.0,222.0,242.0,NaN,0.511176,0.286451,0.090908,47.415927,8.085840,0.0,...,686.0,2.234259,5.89,14.0,9.0,308.0,331.0,1420000.0,5.0,terrace-house
2,93.0,NaN,NaN,2.0,0.163362,0.095877,0.001911,47.397416,8.043150,0.0,...,10149.0,3.549010,6.05,37.0,3092.0,30364.0,33493.0,720000.0,5.0,penthouse
3,154.0,370.0,257.0,NaN,0.511176,0.286451,0.090908,47.415927,8.085840,0.0,...,686.0,2.234259,5.89,14.0,9.0,308.0,331.0,1430000.0,5.0,detached-house
4,142.0,NaN,NaN,0.0,0.333865,0.279276,0.145835,47.404870,8.052781,0.0,...,2638.0,1.708126,6.30,65.0,349.0,941.0,1355.0,995000.0,5.0,flat


### First part of imputation

### Divide categorical "type" variable into dummy variables

In [56]:
clean_df["type_unified"].unique()

array(['penthouse', 'terrace-house', 'detached-house', 'flat',
       'stepped-house', 'farmhouse', 'semi-detached-house',
       'stepped-apartment', 'duplex-maisonette', 'attic-flat', 'loft',
       'chalet', 'villa', 'attic-room', 'secondary-suite', 'castle',
       'detached-secondary-suite', 'studio',
       'furnished-residential-property', 'rustico', 'single-room'],
      dtype=object)

In [57]:
clean_df = pd.get_dummies(clean_df, columns=['type_unified'])

### How many NAN's has each column

In [58]:
(clean_df.isnull().sum() / len(clean_df)) * 100

#Floor und Floorspace????

Living space                       2.668920
Plot area                         63.480272
Floor space                       74.609670
Floor                             54.383702
ForestDensityL                     0.000000
                                    ...    
type_unified_stepped-apartment     0.000000
type_unified_stepped-house         0.000000
type_unified_studio                0.000000
type_unified_terrace-house         0.000000
type_unified_villa                 0.000000
Length: 71, dtype: float64

### Check for every columns if there are absurd values, which are not possible
### e.g. negative prices, negative living space, negative plot area, etc.

In [59]:
stats_df = pd.DataFrame(columns=["min", "0.25 quantile", "mean", "median", "0.75 quantile", "max"])
stats_df["min"] = clean_df.min()
stats_df["0.25 quantile"] = clean_df.quantile(0.25)
stats_df["mean"] = clean_df.mean()
stats_df["median"] = clean_df.median()
stats_df["0.75 quantile"] = clean_df.quantile(0.75)
stats_df["max"] = clean_df.max()
stats_df = stats_df.round(2)
stats_df = stats_df[~stats_df.index.str.startswith("type_")]
stats_df

,min,0.25 quantile,mean,median,0.75 quantile,max
Living space,0.00,97.00,176.42,130.00,185.00,9681.00
Plot area,1.00,347.25,1377.37,646.00,1137.00,350917.00
Floor space,1.00,101.00,201.05,148.00,234.00,7798.00
Floor,-4.00,0.00,4.44,1.00,2.00,1011.00
ForestDensityL,0.00,0.02,0.19,0.11,0.30,0.90
ForestDensityM,0.00,0.00,0.12,0.03,0.19,1.00
ForestDensityS,0.00,0.00,0.08,0.00,0.05,1.00
Latitude,45.83,46.22,46.64,46.50,47.06,47.79
Longitude,5.97,6.93,7.58,7.36,8.29,10.40
NoisePollutionRailwayL,0.00,0.00,0.01,0.00,0.02,0.26


Living space hat einen komischen min Wert

Plot area hat einen komischen min Wert

Floor space Hat einen komischen min Wert

Floor hat komische max Werte

price_cleaned hat einen komischen min Wert

In [60]:
living_space_cutoff = 5
clean_df[clean_df['Living space'] <= living_space_cutoff]

,Living space,Plot area,Floor space,Floor,ForestDensityL,ForestDensityM,ForestDensityS,Latitude,Longitude,NoisePollutionRailwayL,...,type_unified_penthouse,type_unified_rustico,type_unified_secondary-suite,type_unified_semi-detached-house,type_unified_single-room,type_unified_stepped-apartment,type_unified_stepped-house,type_unified_studio,type_unified_terrace-house,type_unified_villa
2710,5.0,1275.0,NaN,NaN,0.022664,0.000000,0.000000,46.749443,6.809191,0.0,...,0,0,0,0,0,0,0,0,0,1
3698,0.0,10000.0,NaN,NaN,0.262601,0.387136,0.151863,46.166641,6.170931,0.0,...,0,0,0,0,0,0,0,0,0,0
11562,1.0,176.0,NaN,NaN,0.496904,0.490431,0.489897,46.080593,7.212619,0.0,...,0,0,0,0,0,0,0,0,0,0
11691,0.0,21000.0,NaN,NaN,0.488697,0.569778,0.489897,46.098200,7.505315,0.0,...,0,0,0,0,0,0,0,0,0,0
15325,5.0,1275.0,NaN,NaN,0.160616,0.051230,0.150340,46.719749,6.784727,0.0,...,0,0,0,0,0,0,0,0,0,1
18303,1.0,176.0,NaN,NaN,0.513901,0.630910,0.613308,46.032303,7.309549,0.0,...,0,0,0,0,0,0,0,0,0,0
21636,1.0,NaN,NaN,NaN,0.728738,0.637685,0.555986,46.332592,7.525694,0.0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
plot_area_cutoff = 15
clean_df[clean_df['Plot area'] <= 15]

,Living space,Plot area,Floor space,Floor,ForestDensityL,ForestDensityM,ForestDensityS,Latitude,Longitude,NoisePollutionRailwayL,...,type_unified_penthouse,type_unified_rustico,type_unified_secondary-suite,type_unified_semi-detached-house,type_unified_single-room,type_unified_stepped-apartment,type_unified_stepped-house,type_unified_studio,type_unified_terrace-house,type_unified_villa
13468,155.0,12.0,189.0,3.0,0.103021,0.096711,0.051118,46.552803,6.597701,0.000000,...,0,0,0,0,0,0,0,0,0,0
13634,102.0,8.0,110.0,0.0,0.018306,0.000000,0.000000,46.639423,6.633247,0.000000,...,0,0,0,0,0,0,0,0,0,0
15059,86.0,15.0,NaN,NaN,0.095162,0.097193,0.153314,46.240700,6.146938,0.068045,...,0,0,0,0,0,0,0,0,0,0
15528,76.0,10.0,78.0,0.0,0.178318,0.126173,0.032757,46.741386,6.884060,0.005165,...,0,0,0,0,0,0,0,0,0,0
16229,60.0,15.0,70.0,0.0,0.000000,0.000000,0.000000,46.726494,7.137549,0.000000,...,0,0,0,0,0,0,0,0,0,0
16522,160.0,10.0,NaN,1.0,0.005083,0.000000,0.000000,46.472305,6.832813,0.039872,...,0,0,0,0,0,0,0,0,0,0
18564,125.0,1.0,NaN,NaN,0.000000,0.000000,0.000000,46.230338,7.364450,0.069983,...,0,0,0,0,0,0,0,0,0,0
18631,130.0,10.0,130.0,4.0,0.000000,0.000000,0.000000,46.230338,7.364450,0.069983,...,0,0,0,0,0,1,0,0,0,0
18632,97.0,10.0,130.0,4.0,0.000000,0.000000,0.000000,46.230338,7.364450,0.069983,...,0,0,0,0,0,0,0,0,0,0
18643,115.0,12.0,142.0,2.0,0.000000,0.000000,0.000000,46.230338,7.364450,0.069983,...,0,0,0,0,0,0,0,0,0,0


In [62]:
floor_space_cutoff = 5
clean_df[clean_df['Floor space'] <= floor_space_cutoff]

,Living space,Plot area,Floor space,Floor,ForestDensityL,ForestDensityM,ForestDensityS,Latitude,Longitude,NoisePollutionRailwayL,...,type_unified_penthouse,type_unified_rustico,type_unified_secondary-suite,type_unified_semi-detached-house,type_unified_single-room,type_unified_stepped-apartment,type_unified_stepped-house,type_unified_studio,type_unified_terrace-house,type_unified_villa
2203,68.0,NaN,4.0,NaN,0.290155,0.249285,0.152095,46.688320,7.844555,0.000488,...,0,0,0,0,0,0,0,0,0,0
4238,74.0,NaN,5.0,NaN,0.390434,0.167669,0.067203,46.808449,9.259200,0.000000,...,0,0,0,0,0,0,0,0,0,0
13210,76.0,NaN,4.0,3.0,0.286686,0.139504,0.000000,47.246667,8.607851,0.000000,...,0,0,0,0,0,0,0,0,0,0
20135,75.0,NaN,4.0,8.0,0.019785,0.000000,0.000000,47.093240,6.814634,0.033235,...,0,0,0,0,0,0,0,0,0,0
22111,195.0,578.0,1.0,NaN,0.298248,0.155929,0.014410,47.332588,8.339168,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [63]:
floor_cutoff = 30
clean_df[clean_df['Floor'] > floor_cutoff]

,Living space,Plot area,Floor space,Floor,ForestDensityL,ForestDensityM,ForestDensityS,Latitude,Longitude,NoisePollutionRailwayL,...,type_unified_penthouse,type_unified_rustico,type_unified_secondary-suite,type_unified_semi-detached-house,type_unified_single-room,type_unified_stepped-apartment,type_unified_stepped-house,type_unified_studio,type_unified_terrace-house,type_unified_villa
3133,96.0,NaN,NaN,100.0,0.459500,0.283642,0.064622,46.559132,7.081714,0.000000,...,0,0,0,0,0,0,0,0,0,0
10857,162.0,NaN,NaN,999.0,0.032747,0.000000,0.000000,46.422879,6.260065,0.001111,...,0,0,0,0,0,0,0,0,0,0
10884,147.0,NaN,NaN,999.0,0.598226,0.324451,0.239227,46.447648,6.133410,0.000000,...,0,0,0,0,0,0,0,0,0,0
11113,159.0,NaN,NaN,999.0,0.005083,0.000000,0.000000,46.472305,6.832813,0.039872,...,0,0,0,0,0,0,0,0,0,0
11308,219.0,NaN,NaN,999.0,0.009171,0.000000,0.000000,46.436880,6.911776,0.033026,...,0,0,0,0,0,0,0,0,0,0
11323,180.0,NaN,NaN,999.0,0.009171,0.000000,0.000000,46.436880,6.911776,0.033026,...,0,0,0,0,0,0,0,0,0,0
13475,97.0,NaN,NaN,999.0,0.103021,0.096711,0.051118,46.552803,6.597701,0.000000,...,0,0,0,0,0,0,0,0,0,0
14062,162.0,NaN,NaN,999.0,0.000000,0.000000,0.000000,46.423691,6.265225,0.021231,...,0,0,0,0,0,0,0,0,0,0
14083,96.0,NaN,NaN,999.0,0.000000,0.000000,0.000000,46.201506,6.146503,0.000000,...,0,0,0,0,0,0,0,0,0,0
14087,97.0,NaN,NaN,999.0,0.000000,0.000000,0.000000,46.201506,6.146503,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [64]:
price_cleaned_cutoff = 15_000
clean_df[clean_df['price_cleaned'] < price_cleaned_cutoff]

,Living space,Plot area,Floor space,Floor,ForestDensityL,ForestDensityM,ForestDensityS,Latitude,Longitude,NoisePollutionRailwayL,...,type_unified_penthouse,type_unified_rustico,type_unified_secondary-suite,type_unified_semi-detached-house,type_unified_single-room,type_unified_stepped-apartment,type_unified_stepped-house,type_unified_studio,type_unified_terrace-house,type_unified_villa
3614,NaN,NaN,NaN,NaN,0.061377,0.065835,0.000000,46.179057,6.119311,0.005745,...,0,0,0,0,0,0,0,0,0,0
3811,NaN,NaN,NaN,NaN,0.034854,0.025740,0.000000,46.262419,6.214042,0.000000,...,0,0,0,0,0,0,0,0,0,0
3906,140.0,NaN,NaN,NaN,0.262601,0.387136,0.151863,46.166641,6.170931,0.000000,...,0,0,0,0,0,0,0,0,0,0
4398,75.0,NaN,NaN,NaN,0.174940,0.091614,0.000000,47.406451,7.040699,0.000000,...,0,0,0,0,0,0,0,0,0,0
5021,45.0,NaN,NaN,8.0,0.161326,0.300538,0.280474,47.053017,6.753855,0.000000,...,0,0,0,0,0,0,0,0,0,0
5026,110.0,NaN,NaN,2.0,0.161326,0.300538,0.280474,47.053017,6.753855,0.000000,...,0,0,0,0,0,0,0,0,0,0
10213,170.0,NaN,NaN,1.0,0.024808,0.028412,0.000000,46.522587,6.635715,0.000000,...,0,0,0,0,0,0,0,0,0,0
10462,82.0,NaN,NaN,NaN,0.197137,0.227047,0.012352,46.517128,6.676822,0.020723,...,1,0,0,0,0,0,0,0,0,0
10530,19.0,NaN,NaN,NaN,0.016314,0.007802,0.000000,46.529557,6.560561,0.001126,...,0,0,0,0,1,0,0,0,0,0
10690,NaN,NaN,NaN,NaN,0.001743,0.000000,0.000000,46.487053,6.421055,0.000000,...,0,0,0,0,0,0,0,0,1,0


#### Remove rows where the Values are "impossible"  --> Als Gruppe noch besprechen

In [65]:
clean_df = clean_df[(clean_df['Living space'] > living_space_cutoff) | (clean_df['Living space'].isnull())]
clean_df = clean_df[(clean_df['Plot area'] > plot_area_cutoff) | (clean_df['Plot area'].isnull())]
clean_df = clean_df[(clean_df['Floor space'] > floor_space_cutoff) | (clean_df['Floor space'].isnull())]
clean_df = clean_df[(clean_df['Floor'] < floor_cutoff) | (clean_df['Floor'].isnull())]
clean_df = clean_df[(clean_df['price_cleaned'] > price_cleaned_cutoff) | (clean_df['price_cleaned'].isnull())]
clean_df.head()

,Living space,Plot area,Floor space,Floor,ForestDensityL,ForestDensityM,ForestDensityS,Latitude,Longitude,NoisePollutionRailwayL,...,type_unified_penthouse,type_unified_rustico,type_unified_secondary-suite,type_unified_semi-detached-house,type_unified_single-room,type_unified_stepped-apartment,type_unified_stepped-house,type_unified_studio,type_unified_terrace-house,type_unified_villa
0,100.0,NaN,NaN,4.0,0.511176,0.286451,0.090908,47.415927,8.085840,0.0,...,1,0,0,0,0,0,0,0,0,0
1,156.0,222.0,242.0,NaN,0.511176,0.286451,0.090908,47.415927,8.085840,0.0,...,0,0,0,0,0,0,0,0,1,0
2,93.0,NaN,NaN,2.0,0.163362,0.095877,0.001911,47.397416,8.043150,0.0,...,1,0,0,0,0,0,0,0,0,0
3,154.0,370.0,257.0,NaN,0.511176,0.286451,0.090908,47.415927,8.085840,0.0,...,0,0,0,0,0,0,0,0,0,0
4,142.0,NaN,NaN,0.0,0.333865,0.279276,0.145835,47.404870,8.052781,0.0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
#Florspace Temporär rausgenommen
clean_df = clean_df.drop(["Floor space"], axis = 1)
clean_df.head()

,Living space,Plot area,Floor,ForestDensityL,ForestDensityM,ForestDensityS,Latitude,Longitude,NoisePollutionRailwayL,NoisePollutionRailwayM,...,type_unified_penthouse,type_unified_rustico,type_unified_secondary-suite,type_unified_semi-detached-house,type_unified_single-room,type_unified_stepped-apartment,type_unified_stepped-house,type_unified_studio,type_unified_terrace-house,type_unified_villa
0,100.0,NaN,4.0,0.511176,0.286451,0.090908,47.415927,8.085840,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
1,156.0,222.0,NaN,0.511176,0.286451,0.090908,47.415927,8.085840,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2,93.0,NaN,2.0,0.163362,0.095877,0.001911,47.397416,8.043150,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
3,154.0,370.0,NaN,0.511176,0.286451,0.090908,47.415927,8.085840,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,142.0,NaN,0.0,0.333865,0.279276,0.145835,47.404870,8.052781,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


### Imputation

In [67]:
clean_df = clean_df[clean_df.price_cleaned.notna()]

In [68]:
nan_percentage = (clean_df.isnull().sum() / len(clean_df)) * 100

cols = nan_percentage[(nan_percentage < 20) & (nan_percentage > 0)].index
cols

Index(['Living space', 'gde_politics_cvp', 'gde_politics_fdp',
       'gde_politics_gps', 'gde_politics_sp', 'gde_politics_svp', 'rooms'],
      dtype='object')

In [69]:
TYPES_WITH_NO_FLOOR = ['type_unified_villa', 'type_unified_farmhouse', 'type_unified_castle', 'type_unified_chalet', 'type_unified_detached-house', 'type_unified_semi-detached-house']

for col in TYPES_WITH_NO_FLOOR:
    clean_df.loc[clean_df[col] == 1., 'Floor'] = clean_df[clean_df[col] == 1.].Floor.fillna(0.)

clean_df[clean_df[TYPES_WITH_NO_FLOOR[2]] == 1.]

,Living space,Plot area,Floor,ForestDensityL,ForestDensityM,ForestDensityS,Latitude,Longitude,NoisePollutionRailwayL,NoisePollutionRailwayM,...,type_unified_penthouse,type_unified_rustico,type_unified_secondary-suite,type_unified_semi-detached-house,type_unified_single-room,type_unified_stepped-apartment,type_unified_stepped-house,type_unified_studio,type_unified_terrace-house,type_unified_villa
2612,NaN,NaN,0.0,0.025704,0.000000,0.000000,47.466193,7.814191,0.041077,0.000443,...,0,0,0,0,0,0,0,0,0,0
3570,900.0,7623.0,0.0,0.275512,0.156081,0.210352,46.496443,6.871439,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
9911,275.0,302.0,0.0,0.000167,0.000000,0.000000,46.593586,6.569423,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
13617,275.0,302.0,0.0,0.000801,0.000000,0.000000,46.593945,6.565308,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
15755,900.0,7623.0,0.0,0.039354,0.034158,0.000000,46.510841,6.849226,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
21516,370.0,1070.0,0.0,0.153606,0.034261,0.003153,46.286171,7.505000,0.000050,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [70]:
type_columns = [col for col in clean_df.columns if col.startswith('type_')]

imputers = {}
df_no_price = clean_df.copy().drop('price_cleaned', axis=1)

columns = df_no_price.columns

for col in type_columns:
    imputer = KNNImputer(n_neighbors=15, weights='distance', keep_empty_features=True)
    #imputer = SimpleImputer(strategy='median', keep_empty_features=True)

    df_no_price.loc[df_no_price[col] == 1., columns] = imputer.fit_transform(df_no_price[df_no_price[col] == 1.])
    imputers[col] = imputer

base_imputer = SimpleImputer(strategy='median', keep_empty_features=True)

df_no_price[columns] = base_imputer.fit_transform(df_no_price)

df_no_price.insert(2, 'price_cleaned', clean_df.price_cleaned.values)
clean_df = df_no_price
clean_df.head()

,Living space,Plot area,price_cleaned,Floor,ForestDensityL,ForestDensityM,ForestDensityS,Latitude,Longitude,NoisePollutionRailwayL,...,type_unified_penthouse,type_unified_rustico,type_unified_secondary-suite,type_unified_semi-detached-house,type_unified_single-room,type_unified_stepped-apartment,type_unified_stepped-house,type_unified_studio,type_unified_terrace-house,type_unified_villa
0,100.0,1282.323307,1150000.0,4.00,0.511176,0.286451,0.090908,47.415927,8.085840,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,156.0,222.000000,1420000.0,2.75,0.511176,0.286451,0.090908,47.415927,8.085840,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,93.0,1198.982216,720000.0,2.00,0.163362,0.095877,0.001911,47.397416,8.043150,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,154.0,370.000000,1430000.0,0.00,0.511176,0.286451,0.090908,47.415927,8.085840,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142.0,462.537377,995000.0,0.00,0.333865,0.279276,0.145835,47.404870,8.052781,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
target_path = Path('./data/model/01_clean_data.pkl')
dump({
    'dataset': clean_df,
    'base_imputer': base_imputer,
    'imputers': imputers
}, target_path)

['data/model/01_clean_data.pkl']